# 30 Ocak - 14 Mart arası COVID-19 Vakalarının incelenmesi

## Gerekli Kütüphaneler

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px

### Verilerin alınması

In [36]:
df = pd.read_csv('covid_19_data .csv',parse_dates=['Last Update'])
df.rename(columns={'ObservationDate':'Date', 'Country/Region':'Country'}, inplace=True)

df_confirmed = pd.read_csv("time_series_covid_19_confirmed .csv")
df_recovered = pd.read_csv("time_series_covid_19_recovered .csv")
df_deaths = pd.read_csv("time_series_covid_19_deaths.csv")

df_confirmed.rename(columns={'Country/Region':'Country'}, inplace=True)
df_recovered.rename(columns={'Country/Region':'Country'}, inplace=True)
df_deaths.rename(columns={'Country/Region':'Country'}, inplace=True)

In [37]:
print("COVID-19 DATA\n")
df.head()

COVID-19 DATA



,SNo,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0


In [38]:
print("COVID-19 Confirmed DATA\n")
df_confirmed.head()

COVID-19 Confirmed DATA



,Province/State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,47,48,50,50,50,53,59,70,75,82
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,360,420,461,502,511,581,639,639,701,773
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,117,130,138,150,150,160,178,178,200,212
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,50,83,93,99,117,129,149,149,197,238


In [39]:
print("COVID-19 Deaths DATA\n")
df_deaths.head()

COVID-19 Deaths DATA



,Province/State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,6,6,6,6,10,10,15,16,19,22
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
print("COVID-19 Recovered DATA\n")
df_recovered.head()

COVID-19 Recovered DATA



,Province/State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,2,2,...,31,31,31,31,31,33,34,34,35,35
1,NaN,Japan,36.0000,138.0000,0,0,0,0,1,1,...,43,46,76,76,76,101,118,118,118,118
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,78,78,78,78,78,78,96,96,97,105
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,22,22,23,24,24,24,26,26,26,35


In [41]:
df.describe()

,SNo,Confirmed,Deaths,Recovered
count,5632.000000,5632.000000,5632.000000,5632.000000
mean,2816.500000,582.415305,18.335760,213.634766
std,1625.962689,4935.578984,195.698066,2363.112194
min,1.000000,0.000000,0.000000,0.000000
25%,1408.750000,2.000000,0.000000,0.000000
50%,2816.500000,10.000000,0.000000,0.000000
75%,4224.250000,94.000000,1.000000,12.000000
max,5632.000000,67790.000000,3075.000000,52960.000000


In [42]:
df.tail()

,SNo,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
5627,5628,03/14/2020,"Virgin Islands, U.S.",US,2020-03-14 16:15:18,1.0,0.0,0.0
5628,5629,03/14/2020,Gibraltar,UK,2020-03-14 16:33:03,1.0,0.0,1.0
5629,5630,03/14/2020,Diamond Princess cruise ship,Australia,2020-03-14 02:33:04,0.0,0.0,0.0
5630,5631,03/14/2020,West Virginia,US,2020-03-10 02:33:04,0.0,0.0,0.0
5631,5632,03/14/2020,NaN,occupied Palestinian territory,2020-03-11 20:53:02,0.0,0.0,0.0


### Vakaların Tarih ve Ülkeye göre gruplanması

In [43]:
df2 = df.groupby(['Date','Country'])[['Date', 'Country', 'Confirmed', 'Deaths', 'Recovered']].sum().reset_index()
df2.tail()

,Date,Country,Confirmed,Deaths,Recovered
2582,03/14/2020,United Arab Emirates,85.0,0.0,17.0
2583,03/14/2020,Uruguay,4.0,0.0,0.0
2584,03/14/2020,Venezuela,2.0,0.0,0.0
2585,03/14/2020,Vietnam,53.0,0.0,16.0
2586,03/14/2020,occupied Palestinian territory,0.0,0.0,0.0


### Türkiye

In [44]:
df.query('Country == "Turkey"').groupby('Date')[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

,Date,Confirmed,Deaths,Recovered
0,03/11/2020,1.0,0.0,0.0
1,03/12/2020,1.0,0.0,0.0
2,03/13/2020,5.0,0.0,0.0
3,03/14/2020,5.0,0.0,0.0


### Çin

In [45]:
df.query('Country == "Mainland China"').groupby('Date')[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

,Date,Confirmed,Deaths,Recovered
0,01/22/2020,547.0,17.0,28.0
1,01/23/2020,639.0,18.0,30.0
2,01/24/2020,916.0,26.0,36.0
3,01/25/2020,1399.0,42.0,39.0
4,01/26/2020,2062.0,56.0,49.0
5,01/27/2020,2863.0,82.0,58.0
6,01/28/2020,5494.0,131.0,101.0
7,01/29/2020,6070.0,133.0,120.0
8,01/30/2020,8124.0,171.0,135.0
9,01/31/2020,9783.0,213.0,214.0


### Ülke Bazlı

In [46]:
df.groupby('Country')[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

,Country,Confirmed,Deaths,Recovered
0,Azerbaijan,1.0,0.0,0.0
1,"('St. Martin',)",2.0,0.0,0.0
2,Afghanistan,58.0,0.0,0.0
3,Albania,118.0,4.0,0.0
4,Algeria,238.0,6.0,28.0
...,...,...,...,...
151,Uruguay,4.0,0.0,0.0
152,Vatican City,4.0,0.0,0.0
153,Venezuela,2.0,0.0,0.0
154,Vietnam,798.0,0.0,438.0


### Tarih Bazlı

In [47]:
df.groupby("Date").sum()

,SNo,Confirmed,Deaths,Recovered
Date,,,,
01/22/2020,741,555.0,17.0,28.0
01/23/2020,2829,653.0,18.0,30.0
01/24/2020,4305,941.0,26.0,36.0
01/25/2020,6490,1438.0,42.0,39.0
01/26/2020,9071,2118.0,56.0,52.0
01/27/2020,12342,2927.0,82.0,61.0
01/28/2020,15262,5578.0,131.0,107.0
01/29/2020,18711,6165.0,133.0,126.0
01/30/2020,23345,8235.0,171.0,143.0


# Görselleştirme
### Tarih bazlı vaka tespit/ ölün/ tedavi edilenlerin gruplanması

In [48]:
confirmed = df.groupby("Date").sum()['Confirmed'].reset_index()
deaths = df.groupby("Date").sum()['Deaths'].reset_index()
recovered = df.groupby("Date").sum()['Recovered'].reset_index()

### Barplot

In [49]:
fig = go.Figure()
fig.add_trace(go.Bar(x = confirmed['Date'], y = confirmed['Confirmed'], name = 'Confirmed', marker_color = 'blue'))
fig.add_trace(go.Bar(x = deaths['Date'], y = deaths['Deaths'], name = 'Deaths', marker_color = 'Red'))
fig.add_trace(go.Bar(x = recovered['Date'], y = recovered['Recovered'], name = 'Recovered', marker_color = 'Green'))
fig.update_layout(title='Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Bar Chart)',
xaxis_tickfont_size=14,
yaxis=dict(
title='Number of Cases',
titlefont_size=16,
tickfont_size=14,),
legend=dict(
x=0,y=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'),barmode='group',
bargap=0.15, # gap between bars of adjacent location coordinates.
bargroupgap=0.1 # gap between bars of the same location coordinate.
)
#fig.show()

plotly.offline.plot(fig, filename= "barplot" + ".html")

'barplot.html'

### Scatterplot

In [50]:
fig.add_trace(go.Scatter(x=confirmed['Date'],
y=confirmed['Confirmed'],
mode='lines+markers',
name='Confirmed',
line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=deaths['Date'],
y=deaths['Deaths'],
mode='lines+markers',
name='Deaths',line=dict(color='Red', width=2)))
fig.add_trace(go.Scatter(x=recovered['Date'],
y=recovered['Recovered'],
mode='lines+markers',name='Recovered',
line=dict(color='Green', width=2)))
fig.update_layout(title='Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Line Chart)'
,xaxis_tickfont_size=14,yaxis=dict(
title='Number of Cases',titlefont_size=16,tickfont_size=14,),legend=dict(x=0,y=1.0,
bgcolor='rgba(255, 255, 255, 0)',
bordercolor='rgba(255, 255, 255, 0)'))
#fig.show()
plotly.offline.plot(fig, filename= "scatter" + ".html")

'scatter.html'

# Harita üzerinden

In [51]:
df_confirmed = df_confirmed[['Province/State', 'Lat', 'Long', 'Country']]
df_latlong = pd.merge(df, df_confirmed, on = ['Province/State', 'Country'])
fig = px.density_mapbox(df_latlong, lat='Lat', lon='Long', hover_name='Province/State', hover_data = ['Confirmed', 'Deaths', 'Recovered'], animation_frame="Date", color_continuous_scale='Rainbow', radius=7, zoom=0, height=700)
fig.update_layout(title='Worldwide Corona Virus Cases Time Lapse - Confirmed, Deaths, Recovered',font=dict(family="Courier New, monospace",size=18,color="#7f7f7f"))
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=0)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
plotly.offline.plot(fig, filename= "harita_" + ".html")

'harita_.html'

### Ülke bazlı

In [52]:
confirmed = df2.groupby(['Date', 'Country']).sum()[['Confirmed']].reset_index()
deaths = df2.groupby(['Date', 'Country']).sum()[['Deaths']].reset_index()
recovered = df2.groupby(['Date', 'Country']).sum()[['Recovered']].reset_index()
latest_date = confirmed['Date'].max()
latest_date

'03/14/2020'

In [53]:
confirmed = confirmed[(confirmed["Date"]==latest_date)][['Country', 'Confirmed']]
deaths = deaths[(deaths["Date"]==latest_date)][['Country', 'Deaths']]
recovered = recovered[(recovered["Date"]==latest_date)][['Country', 'Recovered']]
fig = go.Figure()
fig.add_trace(go.Bar(x = confirmed['Country'], y = confirmed['Confirmed'], name = 'Confirmed', marker_color = 'blue'))
fig.add_trace(go.Bar(x = deaths['Country'], y = deaths['Deaths'], name = 'Deaths', marker_color = 'Red'))
fig.add_trace(go.Bar(x = recovered['Country'], y = recovered['Recovered'], name = 'Recovered', marker_color = 'Green'))
fig.update_layout(title='Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Bar Chart)',xaxis_tickfont_size=14,yaxis=dict(
title='Number of Cases',
titlefont_size=16,tickfont_size=14,),
legend=dict(
x=0,y=1.0,
bgcolor='rgba(255, 255, 255, 0)',
bordercolor='rgba(255, 255, 255, 0)'),
barmode='group',
bargap=0.15, # gap between bars of adjacent location coordinates.
bargroupgap=0.1 # gap between bars of the same location coordinate.
)
plotly.offline.plot(fig, filename= "ulke-bazli" + ".html")

'ulke-bazli.html'

### 14 Mart gününe dek etkilenen tüm ülkeler

In [54]:
all_country = confirmed['Country'].unique()
print("Number of countries with cases: ", str(len(all_country)))
print("Countries with cases: ")
for i in range(len(all_country)):
    print(str(i) + ' - - ' + all_country[i])

Number of countries with cases:  143
Countries with cases: 
0 - - Afghanistan
1 - - Albania
2 - - Algeria
3 - - Andorra
4 - - Antigua and Barbuda
5 - - Argentina
6 - - Armenia
7 - - Aruba
8 - - Australia
9 - - Austria
10 - - Azerbaijan
11 - - Bahrain
12 - - Bangladesh
13 - - Belarus
14 - - Belgium
15 - - Bhutan
16 - - Bolivia
17 - - Bosnia and Herzegovina
18 - - Brazil
19 - - Brunei
20 - - Bulgaria
21 - - Burkina Faso
22 - - Cambodia
23 - - Cameroon
24 - - Canada
25 - - Cayman Islands
26 - - Chile
27 - - Colombia
28 - - Congo (Kinshasa)
29 - - Costa Rica
30 - - Croatia
31 - - Cuba
32 - - Curacao
33 - - Cyprus
34 - - Czech Republic
35 - - Denmark
36 - - Dominican Republic
37 - - Ecuador
38 - - Egypt
39 - - Estonia
40 - - Eswatini
41 - - Ethiopia
42 - - Finland
43 - - France
44 - - French Guiana
45 - - Gabon
46 - - Georgia
47 - - Germany
48 - - Ghana
49 - - Greece
50 - - Guadeloupe
51 - - Guatemala
52 - - Guernsey
53 - - Guinea
54 - - Guyana
55 - - Holy See
56 - - Honduras
57 - - Hungary

### Vakaların haritada gösterimi

In [55]:
import pycountry
countries = {}
for country in pycountry.countries:
   countries[country.name] = country.alpha_3
confirmed["iso_alpha"] = confirmed["Country"].map(countries.get)
deaths["iso_alpha"] = deaths["Country"].map(countries.get)
recovered["iso_alpha"] = recovered["Country"].map(countries.get)
plot_data_confirmed = confirmed[["iso_alpha","Confirmed", "Country"]]
plot_data_deaths = deaths[["iso_alpha","Deaths"]]
plot_data_recovered = recovered[["iso_alpha","Recovered"]]

In [56]:
fig = px.scatter_geo(plot_data_confirmed, locations="iso_alpha", color="Country",
hover_name="iso_alpha", size="Confirmed",
projection="natural earth", title = 'Worldwide Confirmed Cases')
plotly.offline.plot(fig, filename= "vaka-harita" + ".html")

'vaka-harita.html'

### Ölümlerin Haritada gösterimi

In [57]:
fig = px.scatter_geo(plot_data_deaths, locations="iso_alpha", color="Deaths",
hover_name="iso_alpha", size="Deaths",
projection="natural earth", title="Worldwide Death Cases")
plotly.offline.plot(fig, filename= "olumler" + ".html")

'olumler.html'

### Tedavi edilenlerin Haritada Gösterimi

In [58]:
fig = px.scatter_geo(plot_data_recovered, locations="iso_alpha", color="Recovered",
hover_name="iso_alpha", size="Recovered",
projection="natural earth", title="Worldwide Recovered Cases")
plotly.offline.plot(fig, filename= "tedavi" + ".html")

'tedavi.html'

### Dünya Çapında Vakaların Büyüme oranı, ölüm oranı ve tedavi oranı 

In [59]:
world_df = df.groupby(['Date']).agg({'Confirmed': ['sum'], 'Recovered': ['sum'], 'Deaths': ['sum']}).reset_index()
world_df.columns = world_df.columns.get_level_values(0)
world_df['Confirmed Change'] = world_df['Confirmed'].diff().shift(-1)
world_df['Mortality Rate'] = world_df['Deaths'] / world_df['Confirmed']
world_df['Recovery Rate'] = world_df['Recovered'] / world_df['Confirmed']
world_df['Growth Rate'] = world_df['Confirmed Change'] / world_df['Confirmed']
world_df['Growth Rate Prev'] = world_df['Growth Rate'].shift(-1)
world_df['Growth Rate Delta'] = (world_df['Growth Rate Prev'] - world_df['Growth Rate']) / world_df['Growth Rate']

In [60]:
world_df.head(15)

,Date,Confirmed,Recovered,Deaths,Confirmed Change,Mortality Rate,Recovery Rate,Growth Rate,Growth Rate Prev,Growth Rate Delta
0,01/22/2020,555.0,28.0,17.0,98.0,0.030631,0.050450,0.176577,0.441041,1.497734
1,01/23/2020,653.0,30.0,18.0,288.0,0.027565,0.045942,0.441041,0.528162,0.197533
2,01/24/2020,941.0,36.0,26.0,497.0,0.027630,0.038257,0.528162,0.472879,-0.104670
3,01/25/2020,1438.0,39.0,42.0,680.0,0.029207,0.027121,0.472879,0.381964,-0.192258
4,01/26/2020,2118.0,52.0,56.0,809.0,0.026440,0.024551,0.381964,0.905706,1.371180
5,01/27/2020,2927.0,61.0,82.0,2651.0,0.028015,0.020840,0.905706,0.105235,-0.883809
6,01/28/2020,5578.0,107.0,131.0,587.0,0.023485,0.019183,0.105235,0.335766,2.190639
7,01/29/2020,6165.0,126.0,133.0,2070.0,0.021573,0.020438,0.335766,0.205222,-0.388796
8,01/30/2020,8235.0,143.0,171.0,1690.0,0.020765,0.017365,0.205222,0.212897,0.037399
9,01/31/2020,9925.0,222.0,213.0,2113.0,0.021461,0.022368,0.212897,0.394501,0.853015


In [61]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.add_trace(go.Scatter(x=world_df['Date'],
y=world_df['Mortality Rate'],
mode='lines+markers',
name='Mortality rate',
line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=world_df['Date'],
y=world_df['Recovery Rate'],
mode='lines+markers',
name='Recovery rate',
line=dict(color='Green', width=2)))
fig.add_trace(go.Scatter(x=world_df['Date'],
y=world_df['Growth Rate'],
mode='lines+markers',
name='Growth rate confirmed',
line=dict(color='Yellow', width=2)))
fig.update_layout(yaxis=dict(tickformat=".2%"))
plotly.offline.plot(fig, filename= "vaka" + ".html")

'vaka.html'

### Günlük Büyüme Oranındaki Yüzde Değişim

In [62]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')
tmp_df = world_df.copy()
tmp_df = tmp_df[tmp_df['Growth Rate Delta'] < 10]
fig.add_trace(go.Scatter(x=tmp_df['Date'],
y=tmp_df['Growth Rate Delta'],
mode='lines+markers',
name='Growth Acceleration',
line=dict(color='Green', width=3)))
fig.update_layout(yaxis=dict(tickformat=".2%"))
plotly.offline.plot(fig, filename= "artis" + ".html")

'artis.html'

### Değişimin Logaritmik gösterimi

In [63]:
confirmed_by_country_df = df.groupby(['Date', 'Country']).sum().reset_index()
fig = px.line(confirmed_by_country_df, x='Date', y='Confirmed', color='Country', line_group="Country", hover_name="Country")
fig.update_layout(template='plotly_dark',
yaxis_type="log")
plotly.offline.plot(fig, filename= "Logaritmik" + ".html")

'Logaritmik.html'